<a href="https://colab.research.google.com/github/Vizzuality/mangrove-atlas-data/blob/master/process_ee_exports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare data for the mangrove-atlas project

https://github.com/Vizzuality/mangrove-atlas-data

`Edward P. Morris (vizzuality.)`

## Description
This notebook processes earthengine exports and adds them to the mangrove-atlas api.

```
MIT License

Copyright (c) 2020 Vizzuality

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```

# Setup

Instructions for setting up the computing environment.

In [1]:
# Remove sample_data
!rm -r sample_data

## Linux dependencies

Instructions for adding linux (including node, ect.) system packages. 

In [2]:
%%bash
# Install mapshaper CLI (simplification, combining, change format)
npm install -g mapshaper

/tools/node/bin/mapshaper-gui -> /tools/node/lib/node_modules/mapshaper/bin/mapshaper-gui
/tools/node/bin/mapshaper-xl -> /tools/node/lib/node_modules/mapshaper/bin/mapshaper-xl
/tools/node/bin/mapshaper -> /tools/node/lib/node_modules/mapshaper/bin/mapshaper
+ mapshaper@0.5.28
added 49 packages from 75 contributors in 12.579s


## Python packages

In [3]:
!pip install -q geopandas

     |████████████████████████████████| 972kB 5.5MB/s 
     |████████████████████████████████| 14.8MB 297kB/s 
     |████████████████████████████████| 6.5MB 51.8MB/s 


In [4]:
# Show python package versions
#!pip list

## Authorisation

Setting up connections and authorisation to cloud services.

### Google Cloud

This can be done in the URL or via adding service account credentials.

If you do not share the notebook, you can mount your Drive and and transfer credentials to disk. Note if the notebook is shared you always need to authenticate via URL.  

In [5]:
# Set the Google Cloud project id
project_id = "mangrove-atlas-246414"
gc_creds = "mangrove-atlas-246414-2f33cc439deb.json"
username = "edward-morris-vizzuality-com-d@mangrove-atlas-246414.iam.gserviceaccount.com"
gcs_prefix = "gs://mangrove_atlas"

In [6]:
# For auth WITHOUT service account
# https://cloud.google.com/resource-manager/docs/creating-managing-projects
#from google.colab import auth
#auth.authenticate_user()
#!gcloud config set project {project_id}

In [7]:
# If the notebook is shared
#from google.colab import drive
#drive.mount('/content/drive')

In [8]:
# If Drive is mounted, copy GC credentials to home (place in your GDrive, and connect Drive)
!cp "/content/drive/My Drive/{gc_creds}" "/root/.{gc_creds}"

In [9]:
# Auth WITH service account
!gcloud auth activate-service-account {username} --key-file=/root/.{gc_creds} --project={project_id}

Activated service account credentials for: [edward-morris-vizzuality-com-d@mangrove-atlas-246414.iam.gserviceaccount.com]


In [10]:
# Test GC auth
!gsutil ls {gcs_prefix}

gs://mangrove_atlas/wdpa_geometry_types_.csv
gs://mangrove_atlas/./
gs://mangrove_atlas//
gs://mangrove_atlas/analysis/
gs://mangrove_atlas/boundaries/
gs://mangrove_atlas/deforestation-alerts/
gs://mangrove_atlas/ee-export-tables/
gs://mangrove_atlas/ee-upload-manifests/
gs://mangrove_atlas/elevation/
gs://mangrove_atlas/environmental-pressures/
gs://mangrove_atlas/gadm-eez.zarr/
gs://mangrove_atlas/land-cover/
gs://mangrove_atlas/mangrove-properties/
gs://mangrove_atlas/orthoimagery/
gs://mangrove_atlas/physical-environment/
gs://mangrove_atlas/tilesets/
gs://mangrove_atlas/tmp/


# Utils

Generic helper functions used in the subsequent processing. For easy navigation each function seperated into a section with the function name.

## copy_gcs

In [11]:
import os
import subprocess

def copy_gcs(source_list, dest_list, opts=""):
  """
  Use gsutil to copy each corresponding item in source_list
  to dest_list.

  Example:
  copy_gcs(["gs://my-bucket/data-file.csv"], ["."])

  """
  for s, d  in zip(source_list, dest_list):
    cmd = f"gsutil -m cp -r {opts} {s} {d}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
  print("Finished copy")

## list_paths

In [12]:
import glob
import subprocess

def list_paths(uri_prefix, dir_path, file_pattern="*", gsutil=True, return_dir_path=True):
        ''' Creates a list of full paths 
    
        Uses glob regex rules allowing flexible patterns
    
        Parameters
        ----------
        uri_prefix : str
            The (GCS) uri prefix.
        dir_path : str
            Directory path, can use regex.
        file_pattern : str
            File pattern for glob searching.
        gsutil : bool
            Use gsutil, default is True.
        return_dir_path : bool
            Return directory path relative to uri_prefix, default is True.        
    
        Returns
        -------
        List of path strings.
        
        Examples
        --------
        # Requires authentication
        #list_paths("gs://skydipper-water-quality", "cloud-masks/*", "*.tif", True, False)
        '''
        p = f"{uri_prefix}/{dir_path}/{file_pattern}"
        print(f"\nSearching {p}\n")
        if not gsutil:
          out = glob.glob(p)
        if gsutil:
          cmd = f"gsutil ls {p}"
          out = subprocess.check_output(cmd, shell=True).decode('utf8').split('\n')
          out.pop(-1)
        if return_dir_path:
          out = [f.split(uri_prefix)[1] for f in out]  
        print(f"\nFound {len(out)} path(s)\n")
        return out

#list_paths("gs://mangrove_atlas", "land-cover/gmw1996v2.0", file_pattern="*.tif", gsutil=True, return_dir_path=False)

## join_geojson

In [13]:
def join_geojson(src_path, join_path, keys, dest_path):
  """
  Use mapshaper to join two geojson by keys.

  Example:
  join_geojson("source.geojson", basename)

  """
  cmd = f"mapshaper-xl -i {src_path} -join {join_path} keys={keys} -o {dest_path} force"
  print(f"Processing: {cmd}")
  r = 1
  r = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  return r.stderr

## combine_geojson

In [14]:
import subprocess

def combine_geojson(src_path, dest_path):
  """
  Use mapshaper to combine geojson in src_path
  to a single geojson file dest_path.

  Example:
  combine_geojson("./geojsons", basename)

  """
  cmd = f"mapshaper-xl -i {src_path} combine-files -merge-layers force -o {dest_path} force"
  print(f"Processing: {cmd}")
  r = 1
  r = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  return r.stderr


# Processing

Data processing organised into sections.

# Get GEE analysis output

+ https://code.earthengine.google.com/?accept_repo=users/vizzuality/mangrove-atlas

In [15]:
# Get the table of locations
copy_gcs(["gs://mangrove_atlas/ee-export-tables/locations_aoi-country-wdpa_20190924.geojson"], ["."])

Processing: gsutil -m cp -r  gs://mangrove_atlas/ee-export-tables/locations_aoi-country-wdpa_20190924.geojson .
Task created
Finished copy


In [16]:
# View
import geopandas as gpd

locations = gpd.read_file("locations_aoi-country-wdpa_20190924.geojson")
locations

,area_m2,bounds,coast_length_m,designation_eng,id,iso,iucn_cat,name,perimeter_m,type,geometry
0,5.179986e+09,"{'coordinates': [[[39.179499025203995, -8.6471...",299548.67,None,1_1_1_00000000000000000000,TZA,None,Rufiji Delta,5.126638e+05,aoi,"POLYGON ((39.17950 -8.16171, 39.26374 -8.41978..."
1,4.035933e+09,"{'coordinates': [[[-16.84288978182007, 13.4842...",383936.07,None,1_1_2_00000000000000000000,SEN,None,Saloum Delta,2.577952e+05,aoi,"POLYGON ((-16.84289 13.48422, -16.59811 13.484..."
2,4.980372e+11,"{'coordinates': [[[-8.606381596274959, 1.02165...",1302793.70,None,1_2_0,CIV,None,Cote d'Ivoire,3.649215e+06,country,"POLYGON ((-8.60638 6.50782, -8.54410 6.49215, ..."
3,4.812580e+11,"{'coordinates': [[[8.324793879943034, 1.654165...",784974.38,None,1_2_1,CMR,None,Cameroon,4.517606e+06,country,"POLYGON ((8.32479 4.06092, 8.35139 4.07500, 8...."
4,1.669739e+11,"{'coordinates': [[[41.93244097943424, -14.4272...",342264.73,None,1_2_2,COM,None,Comoros,1.727243e+06,country,"POLYGON ((41.93244 -11.92195, 41.93735 -12.116..."
...,...,...,...,...,...,...,...,...,...,...,...
257,6.896360e+06,"{'coordinates': [[[-68.32896814629446, 12.1459...",0,"Ramsar Site, Wetland of International Importance",2_000000000000000002d0,BES,Not Reported,Klein Bonaire Island & adjacent sea,1.072414e+04,wdpa,"POLYGON ((-68.32897 12.15228, -68.32889 12.151..."
258,8.286469e+06,"{'coordinates': [[[-68.24361649429329, 12.0865...",4440.15,"Ramsar Site, Wetland of International Importance",2_000000000000000002d1,BES,Not Reported,Het Lac,1.310248e+04,wdpa,"POLYGON ((-68.24362 12.09312, -68.24356 12.090..."
259,5.597392e+06,"{'coordinates': [[[-68.28155002352369, 12.0251...",3532.64,"Ramsar Site, Wetland of International Importance",2_000000000000000002d2,BES,Not Reported,Het Pekelmeer,1.986232e+04,wdpa,"POLYGON ((-68.28155 12.06170, -68.28141 12.060..."
260,1.257184e+06,"{'coordinates': [[[45.25733403710994, -12.7850...",1002.12,"Ramsar Site, Wetland of International Importance",2_000000000000000005bf,MYT,Not Reported,La Vasière des Badamiers,6.030065e+03,wdpa,"POLYGON ((45.25733 -12.78164, 45.25738 -12.781..."


In [17]:
# Extract id list
ids = locations['id'].drop_duplicates().to_list()
print('Number of locations:', len(ids))

Number of locations: 262


In [18]:
# Get analysis directories
dl = ['land-cover', 'mangrove-properties', 'environmental-pressures', 'mangrove-carbon', 'length-coast-cover']
pl = [f"{gcs_prefix}/ee-export-tables/{d}" for d in dl]

copy_gcs(pl, ["." for d in dl])

Processing: gsutil -m cp -r  gs://mangrove_atlas/ee-export-tables/land-cover .
Task created
Processing: gsutil -m cp -r  gs://mangrove_atlas/ee-export-tables/mangrove-properties .
Task created
Processing: gsutil -m cp -r  gs://mangrove_atlas/ee-export-tables/environmental-pressures .
Task created
Processing: gsutil -m cp -r  gs://mangrove_atlas/ee-export-tables/mangrove-carbon .
Task created
Processing: gsutil -m cp -r  gs://mangrove_atlas/ee-export-tables/length-coast-cover .
Task created
Finished copy


In [19]:
# Get file names from each dir
import os
fl = [list_paths("/content", d, file_pattern="*.geojson", gsutil=False, return_dir_path=True) for d in dl]
flb = [[os.path.basename(fp).split(".")[0] for fp in f] for f in fl]


Searching /content/land-cover/*.geojson


Found 262 path(s)


Searching /content/mangrove-properties/*.geojson


Found 262 path(s)


Searching /content/environmental-pressures/*.geojson


Found 262 path(s)


Searching /content/mangrove-carbon/*.geojson


Found 262 path(s)


Searching /content/length-coast-cover/*.geojson


Found 261 path(s)



In [20]:
print("Missing ids land-cover:")
print(set(ids).difference(set(flb[0])))

print("\nMissing ids mangrove-properties:")
print(set(ids).difference(set(flb[1])))

print("\nMissing ids environmental-pressures:")
print(set(ids).difference(set(flb[2])))

print("\nMissing ids mangrove-carbon:")
print(set(ids).difference(set(flb[3])))

print("\nMissing ids length-coast-cover:")
print(set(ids).difference(set(flb[4])))

Missing ids land-cover:
set()

Missing ids mangrove-properties:
set()

Missing ids environmental-pressures:
set()

Missing ids mangrove-carbon:
set()

Missing ids length-coast-cover:
{'2_00000000000000000793'}


## Combine into single geojson

In [21]:
# Combine geojsons and convert to geojson
!mkdir combined
for d in dl:
  combine_geojson(f"./{d}/*.geojson", f"./combined/{d}.geojson")

Processing: mapshaper-xl -i ./land-cover/*.geojson combine-files -merge-layers force -o ./combined/land-cover.geojson force
Processing: mapshaper-xl -i ./mangrove-properties/*.geojson combine-files -merge-layers force -o ./combined/mangrove-properties.geojson force
Processing: mapshaper-xl -i ./environmental-pressures/*.geojson combine-files -merge-layers force -o ./combined/environmental-pressures.geojson force
Processing: mapshaper-xl -i ./mangrove-carbon/*.geojson combine-files -merge-layers force -o ./combined/mangrove-carbon.geojson force
Processing: mapshaper-xl -i ./length-coast-cover/*.geojson combine-files -merge-layers force -o ./combined/length-coast-cover.geojson force


In [22]:
#!mapshaper-xl -i ./locations/*.geojson combine-files -merge-layers force -o ./combined/locations.geojson force

In [23]:
copy_gcs(["./combined/*.geojson"], [f"{gcs_prefix}/ee-export-tables"])

Processing: gsutil -m cp -r  ./combined/*.geojson gs://mangrove_atlas/ee-export-tables
Task created
Finished copy


In [24]:
for d in dl:
  print(list(gpd.read_file(f"./combined/{d}.geojson").drop(["iso", "name", "type", "geometry"], 1).columns))


['id', 'area_mangrove_gain_m2', 'area_mangrove_loss_m2', 'area_mangrove_m2']
['id', 'agb_mangrove_hist_mgha-1', 'agb_mangrove_mgha-1', 'hba_mangrove_hist_m', 'hba_mangrove_m', 'hmax_mangrove_hist_m', 'hmax_mangrove_m']
['id', 'con_hotspot_summary_km2']
['id', 'agb_tco2e', 'bgb_tco2e', 'soc_tco2e', 'toc_hist_tco2eha-1', 'toc_tco2e']
['id', 'length_coast_m', 'length_mangrove_m']


In [26]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import pprint


# Join geojson by id
out = locations#gpd.read_file("./locations.geojson")
#print(out)
for d in dl:
  out = out.merge(gpd.read_file(f"./combined/{d}.geojson").drop(["geometry", "id"], 1), how='left', on=["iso", "name", "type"])
  #print(out)

# FIXME total length of coast
out.loc[:, 'coast_length_m'] = out.loc[:, 'length_coast_m']
out = out.drop('length_coast_m', axis=1)

# FIXME: gain and loss got mixed up...
# ASSETS ARE NOW FIXED! SO THIS SHOULD NOT BE NEEDED NEXT TIME THE ANALYSIS IS RUN
# Swap gain and loss
print('Loss', out.loc[0, 'area_mangrove_loss_m2'])
print('Gain', out.loc[0, 'area_mangrove_gain_m2'])
out = out.rename(columns={'area_mangrove_gain_m2': 'area_mangrove_loss_m2', 'area_mangrove_loss_m2': 'area_mangrove_gain_m2'})
print('Loss', out.loc[0, 'area_mangrove_loss_m2'])
print('Gain', out.loc[0, 'area_mangrove_gain_m2'])

# FIXME: rmv year 2000 from gain and loss
def rmv_year(df, year='2000', loss_name='area_mangrove_loss_m2', gain_name='area_mangrove_gain_m2'):
  loss = list(df.loc[:,[loss_name]].to_dict().get(loss_name).values())
  gain = list(df.loc[:,[gain_name]].to_dict().get(gain_name).values())
  out_loss = []
  out_gain = []
  # loop through cases
  for l, g in zip(loss, gain):
    nc_loss = {}
    nc_gain = {}
    # loop through years
    if isinstance(l, str):
      l = json.loads(l)
    if isinstance(g, str):
      g = json.loads(g)
    for k, v in l.items():
      if k != year:
        gg = dict({k:dict(g).get(k)})
        nc_gain.update(gg)
        ll = dict({k:v})
        nc_loss.update(ll)
    out_gain.append(nc_gain)
    out_loss.append(nc_loss)
  return out_loss, out_gain     

loss, gain = rmv_year(out)
out.loc[:, 'area_mangrove_gain_m2'] = gain
out.loc[:, 'area_mangrove_loss_m2'] = loss

# FIXME: adjust length_mangrove_m by factor of 0.66, to simulate using a buffer of 200m
# THIS IS CHANGED IN THE CODE SO SHOULD BE REMOVED NEXT TIME CODE IS RUN
def adjust_length(df, dvar='length_mangrove_m'):
  length = list(df.loc[:,[dvar]].to_dict().get(dvar).values())
  print(length)
  out = []
  # loop through cases
  for l in length:
    #print(type(l))
    #print(l)
    nc = {}
    # loop through years
    if isinstance(l, float):
      l = dict({'1996': 0, '2007': 0, '2008': 0, '2009': 0, '2010': 0, '2015': 0, '2016': 0})
    if isinstance(l, str):
      l = json.loads(l)
    for k, v in l.items():
      ll = dict({k: v })
      nc.update(ll)
    out.append(nc)
  return out
print('Length', out.loc[0, 'length_mangrove_m'])
out.loc[:, 'length_mangrove_m'] =  adjust_length(out) 
print('Length adjusted', out.loc[0, 'length_mangrove_m'])

# FIXME: calculate net change...


# FIXME: correct TOC to be only AGB and SOC
# THINK THIS IS FIXED, BUT SHOULD NOT BE AN ISSUE
def fix_toc(df):
  agb = df['agb_tco2e']
  soc = df['soc_tco2e']
  out = []
  for a,s in zip(agb, soc):
    out.append({'2016': a.get('2016') + s.get('2016')})
  return out   
out.loc[:, 'toc_tco2e'] = fix_toc(out)

# Create global summary
# select only countries
countries = out[out['type']=='country']
#print(countries.columns)

# create single series
worldwide = countries.loc[4, :]
print(worldwide.index)
worldwide.loc['id'] = 'worldwide'
worldwide.loc['type'] = 'worldwide'
worldwide.loc['iso'] = 'WORLDWIDE'
worldwide.loc['name'] = 'Worldwide'

# total area of landmass (https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area)
worldwide.loc['area_m2'] = 148940000*1000*1000
# total coast length (https://en.wikipedia.org/wiki/List_of_countries_by_length_of_coastline)
worldwide.loc['coast_length_m'] = 1634701*1000
# set to some cols to NaN
worldwide.loc[('bounds', 'designation_eng','iucn_cat', 'perimeter_m', 'geometry', 'con_hotspot_summary_km2')] = np.nan
# create lists of varible names for each summary type
dvars_sum = ['area_mangrove_gain_m2', 'area_mangrove_loss_m2', 'area_mangrove_m2','agb_tco2e', 'bgb_tco2e','soc_tco2e', 'toc_tco2e', 'length_mangrove_m']
dvars_mean = ['agb_mangrove_mgha-1','hmax_mangrove_m']
dvars_hist = ['agb_mangrove_hist_mgha-1','hmax_mangrove_hist_m']
dvars_hist_2 = 'toc_hist_tco2eha-1'

# define JSON encoder for np
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

# calc sums
def calc_sum(df, var_name):
  return json.dumps(pd.DataFrame(list(df.loc[:,[var_name]].to_dict().get(var_name).values())).sum().to_dict(), cls=NpEncoder)
for dvar in dvars_sum:
  worldwide.loc[dvar] = calc_sum(countries, dvar)

# calc means
def calc_mean(df, var_name):
  out = pd.DataFrame(list(df.loc[:,[var_name]].to_dict().get(var_name).values()))
  out = out.replace(0, np.nan)
  out = out.mean().to_dict()  
  return json.dumps(out, cls=NpEncoder)  
for dvar in dvars_mean:
  worldwide.loc[dvar] = calc_mean(countries, dvar)

# calc histogram list sums
def calc_hist(df, var_name):
  df = list(df.loc[:,[var_name]].to_dict().get(var_name).values())
  df = pd.DataFrame.from_records(df).applymap(lambda x: pd.DataFrame(x)).to_dict()
  #print(df)
  out = {}
  for k, v in df.items():
      keys = []
      vals = []
      for kk, vv in v.items():
        keys.append(list(vv.get(0).values))
        vals.append(list(vv.get(1).values))
      keys = set([item for sublist in keys for item in sublist])
      vals = pd.DataFrame(vals, columns=keys).sum(axis=0).to_list()
      out.update({k: [[k, v] for k,v in zip(keys, vals)]})  
      return json.dumps(out, cls=NpEncoder)  
for dvar in dvars_hist:
  print(dvar)
  worldwide.loc[dvar] = calc_hist(countries, dvar)

# calc histogram dict sums
def calc_hist_2(df, var_name):
  df = list(df.loc[:,[var_name]].to_dict().get(var_name).values())
  df = pd.DataFrame.from_dict(df).to_dict()#.applymap(lambda x: pd.DataFrame.from_dict(x))
  
  # loop for years
  out = {}
  for k, v in df.items():
    keys = []
    vals = []
    # loop for cases
    for kk, vv in v.items():
      keys.append(list(vv.keys()))
      vals.append(list(vv.values()))
    keys = set([item for sublist in keys for item in sublist])
    vals = pd.DataFrame(vals, columns=keys).sum(axis=0).to_list()
    out.update({k: {k:v for k,v in zip(keys, vals)}})
  return json.dumps(out, cls=NpEncoder)
worldwide.loc[dvars_hist_2] = calc_hist_2(countries, dvars_hist_2)  


print(worldwide)

# Add worldwide to data table
out = out.append(worldwide, ignore_index=True)

out.to_file("mangrove-atlas-data.geojson", driver='GeoJSON')
copy_gcs(["mangrove-atlas-data.geojson"],[f"{gcs_prefix}/ee-export-tables"])

Loss {'2007': 2089709.2835571289, '2008': 2085957.6754925898, '2009': 828953.8920288086, '2010': 1689577.5714968215, '2015': 11175942.255300485, '2016': 5931565.107328048}
Gain {'2007': 10497043.97499234, '2008': 1295745.3790718827, '2009': 3126896.42621448, '2010': 3739384.8460834576, '2015': 8005266.729488119, '2016': 7636380.109841979}
Loss {'2007': 10497043.97499234, '2008': 1295745.3790718827, '2009': 3126896.42621448, '2010': 3739384.8460834576, '2015': 8005266.729488119, '2016': 7636380.109841979}
Gain {'2007': 2089709.2835571289, '2008': 2085957.6754925898, '2009': 828953.8920288086, '2010': 1689577.5714968215, '2015': 11175942.255300485, '2016': 5931565.107328048}
Length {'1996': 244180.04, '2007': 242171.93, '2008': 238686.9, '2009': 235701.16, '2010': 233337.21, '2015': 238796.26, '2016': 238776}
[{'1996': 244180.04, '2007': 242171.93, '2008': 238686.9, '2009': 235701.16, '2010': 233337.21, '2015': 238796.26, '2016': 238776}, {'1996': 272547.51, '2007': 276903.79, '2008': 27

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1

hmax_mangrove_hist_m


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


area_m2                                                       148940000000000
bounds                                                                    NaN
coast_length_m                                                     1634701000
designation_eng                                                           NaN
id                                                                  worldwide
iso                                                                 WORLDWIDE
iucn_cat                                                                  NaN
name                                                                Worldwide
perimeter_m                                                               NaN
type                                                                worldwide
geometry                                                                  NaN
area_mangrove_loss_m2       {"2007": 5945759989.874274, "2008": 998112838....
area_mangrove_gain_m2       {"2007": 2069321527.5241818, "2008":

In [27]:
# FIXME: calculate net change...
# THIS IS DONE ON THE API AT THE MOMENT, IN FUTURE WE SHOULD MOVE TO PYTHON
def calc_net_change(df, loss_name='area_mangrove_loss_m2', gain_name='area_mangrove_gain_m2'):
  loss = list(df.loc[:,[loss_name]].to_dict().get(loss_name).values())
  gain = list(df.loc[:,[gain_name]].to_dict().get(gain_name).values())
  out = []
  # loop through cases
  for l, g in zip(loss, gain):
    nc = {}
    # loop through years
    if isinstance(l, str):
      l = json.loads(l)
    if isinstance(g, str):
      g = json.loads(g)
    for k, v in l.items():
      if k != "2000":
        gg = dict(g).get(k)
        change = dict({k: gg - v})
        nc.update(change)
    out.append(nc)
  return out    

#calc_net_change(out)


In [28]:
tst = gpd.read_file("mangrove-atlas-data.geojson")
tst

,area_m2,bounds,coast_length_m,designation_eng,id,iso,iucn_cat,name,perimeter_m,type,area_mangrove_loss_m2,area_mangrove_gain_m2,area_mangrove_m2,agb_mangrove_hist_mgha-1,agb_mangrove_mgha-1,hba_mangrove_hist_m,hba_mangrove_m,hmax_mangrove_hist_m,hmax_mangrove_m,con_hotspot_summary_km2,agb_tco2e,bgb_tco2e,soc_tco2e,toc_hist_tco2eha-1,toc_tco2e,length_mangrove_m,geometry
0,5.179986e+09,"{'coordinates': [[[39.179499025203995, -8.6471...",4.229490e+05,None,1_1_1_00000000000000000000,TZA,None,Rufiji Delta,5.126638e+05,aoi,"{'2007': 10497043.97499234, '2008': 1295745.37...","{'2007': 2089709.2835571289, '2008': 2085957.6...","{'1996': 502876968.289535, '2007': 494449870.5...","{'1996': [[0, 2592.270588235294], [10, 2599.59...","{'1996': 254.83199344353682, '2000': 260.00655...",None,None,"{'1996': [[0, 1143.3019607843137], [1, 508], [...","{'1996': 19.65844454015769, '2000': 19.9133823...",{'LT_advice': {'Benefits From Conservation': 1...,{'2016': 20828348.624156356},{'2016': 16877141.462325037},{'2016': 48807262.43887534},"{'2016': {'0--700': 19064, '1400--2100': 24748...",{'2016': 69635611.0630317},"{'1996': 244180.04, '2007': 242171.93, '2008':...","POLYGON ((39.17950 -8.16171, 39.26374 -8.41978..."
1,4.035933e+09,"{'coordinates': [[[-16.84288978182007, 13.4842...",4.180566e+05,None,1_1_2_00000000000000000000,SEN,None,Saloum Delta,2.577952e+05,aoi,"{'2007': 20752909.708496094, '2008': 3595126.2...","{'2007': 16773765.9163208, '2008': 6758886.722...","{'1996': 574127185.7246239, '2007': 569995349....","{'1996': [[0, 42021], [10, 231221], [20, 0], [...","{'1996': 29.864111940271503, '2000': 28.110423...",None,None,"{'1996': [[0, 13794], [1, 28227], [2, 0], [3, ...","{'1996': 6.4667908140003725, '2000': 5.9640542...",{'LT_advice': {'Benefits From Conservation': 9...,{'2016': 2762126.202612892},{'2016': 2238141.6548900255},{'2016': 63014436.01511347},"{'2016': {'0--700': 29051, '1400--2100': 61577...",{'2016': 65776562.217726365},"{'1996': 272547.51, '2007': 276903.79, '2008':...","POLYGON ((-16.84289 13.48422, -16.59811 13.484..."
2,4.980372e+11,"{'coordinates': [[[-8.606381596274959, 1.02165...",5.401392e+05,None,1_2_0,CIV,None,Cote d'Ivoire,3.649215e+06,country,"{'2007': 1929200.5380249023, '2008': 484475.23...","{'2007': 575303.268737793, '2008': 456064.0272...","{'1996': 62833291.21740723, '2007': 61468709.6...","{'1996': [[0, 7218], [10, 9282], [20, 0], [30,...","{'1996': 92.06579802784864, '2000': 130.599809...",None,None,"{'1996': [[0, 4966], [1, 2252], [2, 0], [3, 40...","{'1996': 12.625459910900535, '2000': 15.614919...",{'LT_advice': {'Benefits From Conservation': 1...,{'2016': 913389.7190793986},{'2016': 740116.6447376397},{'2016': 6336688.030987408},"{'2016': {'0--700': 8761, '1400--2100': 21961,...",{'2016': 7250077.750066807},"{'1996': 50622.72, '2007': 48775.7, '2008': 48...","POLYGON ((-8.60638 6.50782, -8.54410 6.49215, ..."
3,4.812580e+11,"{'coordinates': [[[8.324793879943034, 1.654165...",9.896331e+05,None,1_2_1,CMR,None,Cameroon,4.517606e+06,country,"{'2007': 8430770.725510301, '2008': 7080283.40...","{'2007': 6879491.6271362305, '2008': 7009291.2...","{'1996': 1886049732.1722608, '2007': 188466437...","{'1996': [[0, 35524.46666666667], [10, 51763.6...","{'1996': 203.12028202489412, '2000': 209.11671...",None,None,"{'1996': [[0, 30088.1137254902], [1, 5436.3529...","{'1996': 21.87653862130911, '2000': 22.2943538...",{'LT_advice': {'Benefits From Conservation': 1...,{'2016': 62965321.33934208},{'2016': 51020589.99686585},{'2016': 256570712.85158575},"{'2016': {'0--700': 96755, '1400--2100': 10382...",{'2016': 319536034.1909278},"{'1996': 643090.12, '2007': 641774.46, '2008':...","POLYGON ((8.32479 4.06092, 8.35139 4.07500, 8...."
4,1.669739e+11,"{'coordinates': [[[41.93244097943424, -14.4272...",5.037982e+05,None,1_2_2,COM,None,Comoros,1.727243e+06,country,"{'2007': 0, '2008': 0, '2009': 0, '2010': 1747...","{'2007': 0, '2008': 5243.1217041015625, '2009'...","{'1996': 1054134.1912231445, '2007': 1054134.1...","{'

In [29]:
# Validate
tst = gpd.read_file("mangrove-atlas-data.geojson")

# Check all cases are dict objects (returns true if any ARE NOT DICT)
dvars =['area_mangrove_loss_m2', 'area_mangrove_gain_m2', 'area_mangrove_m2', 'agb_mangrove_hist_mgha-1', 'agb_mangrove_mgha-1',
'hba_mangrove_hist_m', 'hba_mangrove_m', 'hmax_mangrove_hist_m', 'hmax_mangrove_m',
'con_hotspot_summary_km2', 'agb_tco2e', 'bgb_tco2e', 'soc_tco2e', 'toc_hist_tco2eha-1', 'toc_tco2e', 'length_mangrove_m']

print('Any cases ARE NOT DICT?\n')
for dvar in dvars:
  print(dvar, tst[dvar].apply(lambda x: not isinstance(x, dict)).values.any())


Any cases ARE NOT DICT?

area_mangrove_loss_m2 False
area_mangrove_gain_m2 False
area_mangrove_m2 False
agb_mangrove_hist_mgha-1 False
agb_mangrove_mgha-1 False
hba_mangrove_hist_m True
hba_mangrove_m True
hmax_mangrove_hist_m False
hmax_mangrove_m False
con_hotspot_summary_km2 True
agb_tco2e False
bgb_tco2e False
soc_tco2e False
toc_hist_tco2eha-1 False
toc_tco2e False
length_mangrove_m False


In [31]:
%%bash
# Add to the API
curl -X "POST" "https://mangrove-atlas-api-staging.herokuapp.com/api/mangrove_data/import_geojson?reset=true" \
     -H "Content-Type: multipart/form-data" \
     -F "file=/content/mangrove-atlas-data.geojson"

{"status":500,"error":"Internal Server Error"}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   221  100    46  100   175     58    222 --:--:-- --:--:-- --:--:--   280


## Convert to flat CSV

NOT USED ANYMORE!!

In [ ]:
# Get gee analysis output
copy_gcs([f"{gcs_prefix}/ee-export-tables/mangrove-atlas-data.geojson"], ["."])

Processing: gsutil -m cp -r  gs://mangrove_atlas/ee-export-tables/mangrove-atlas-data.geojson .
Task created
Finished copy


In [ ]:
# Convert LOCATIONS table from geojson to a flat CSV structure
import json
import pandas as pd

b_file_path = "locations_aoi-country-wdpa_20190924"
data=json.load(open(b_file_path + '.geojson'))['features']

df_list = []
for f in data:
  geometry = f.get('geometry')
  #print(geometry)
  ids = f.get('id')
  props = f.get('properties')
  #print(props.keys())
  bounds = props['bounds']
  del props['bounds']
  #print(props)
  out = pd.DataFrame.from_records([props])
  out['id'] = ids
  out['geometry'] = json.dumps(geometry)
  out['bounds'] = json.dumps(bounds)
  df_list.append(out)
  
df = pd.concat(df_list)
df = df.rename(columns={"type": "location_type", "length_coast_m": "coast_length_m", "id": "location_id"})
#df = df[["name", "location_type", "iso", "bounds", "geometry", "area_m2", "perimeter_m", "coast_length_m", "location_id"]]

df.to_csv(b_file_path + '.csv', index=False, na_rep="NA", sep=";")
df

,area_m2,coast_length_m,designation_eng,location_id,iso,iucn_cat,name,perimeter_m,location_type,geometry,bounds
0,5.179986e+09,299548.67,None,1_1_1_00000000000000000000,TZA,None,Rufiji Delta,5.126638e+05,aoi,"{""coordinates"": [[[39.179499025203995, -8.1617...","{""coordinates"": [[[39.179499025203995, -8.6471..."
0,4.035933e+09,383936.07,None,1_1_2_00000000000000000000,SEN,None,Saloum Delta,2.577952e+05,aoi,"{""coordinates"": [[[-16.84288978182007, 13.4842...","{""coordinates"": [[[-16.84288978182007, 13.4842..."
0,4.980372e+11,1302793.70,None,1_2_0,CIV,None,Cote d'Ivoire,3.649215e+06,country,"{""coordinates"": [[[-8.606381596274959, 6.50781...","{""coordinates"": [[[-8.606381596274959, 1.02165..."
0,4.812580e+11,784974.38,None,1_2_1,CMR,None,Cameroon,4.517606e+06,country,"{""coordinates"": [[[8.324793879943034, 4.060916...","{""coordinates"": [[[8.324793879943034, 1.654165..."
0,1.669739e+11,342264.73,None,1_2_2,COM,None,Comoros,1.727243e+06,country,"{""coordinates"": [[[41.93244097943424, -11.9219...","{""coordinates"": [[[41.93244097943424, -14.4272..."
...,...,...,...,...,...,...,...,...,...,...,...
0,6.896360e+06,0,"Ramsar Site, Wetland of International Importance",2_000000000000000002d0,BES,Not Reported,Klein Bonaire Island & adjacent sea,1.072414e+04,wdpa,"{""coordinates"": [[[-68.32896814629446, 12.1522...","{""coordinates"": [[[-68.32896814629446, 12.1459..."
0,8.286469e+06,4440.15,"Ramsar Site, Wetland of International Importance",2_000000000000000002d1,BES,Not Reported,Het Lac,1.310248e+04,wdpa,"{""coordinates"": [[[-68.24361649429329, 12.0931...","{""coordinates"": [[[-68.24361649429329, 12.0865..."
0,5.597392e+06,3532.64,"Ramsar Site, Wetland of International Importance",2_000000000000000002d2,BES,Not Reported,Het Pekelmeer,1.986232e+04,wdpa,"{""coordinates"": [[[-68.28155002352369, 12.0616...","{""coordinates"": [[[-68.28155002352369, 12.0251..."
0,1.257184e+06,1002.12,"Ramsar Site, Wetland of International Importance",2_000000000000000005bf,MYT,Not Reported,La Vasière des Badamiers,6.030065e+03,wdpa,"{""coordinates"": [[[45.25733403710994, -12.7816...","{""coordinates"": [[[45.25733403710994, -12.7850..."


In [ ]:
# add to GCS
copy_gcs([f"{b_file_path}.csv"], [f"{gcs_prefix}/ee-export-tables"])

Processing: gsutil -m cp -r  locations_aoi-country-wdpa_20190924.csv gs://mangrove_atlas/ee-export-tables
Task created
Finished copy


In [ ]:
df[['location_id']].size == pd.unique(df['location_id']).size

True

In [ ]:
%%bash
# Add to API
curl -X "POST" "https://mangrove-atlas-api.herokuapp.com/api/locations/import?reset=true" \
      -H "Content-Type: multipart/form-data" \
      -F "file=@locations_aoi-country-wdpa_20190924.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9707k    0     0  100 9707k      0   579k  0:00:16  0:00:16 --:--:--     0


In [ ]:
# Get gee analysis output
# FIXME MAKE GEE OUTPUT FILE AUTOMATIC NAMED
b_file_path = "mangrove-atlas-data"
copy_gcs([f"{gcs_prefix}/ee-export-tables/{b_file_path}.geojson"], ["."], )

Processing: gsutil -m cp -r  gs://mangrove_atlas/ee-export-tables/mangrove-atlas-data.geojson .
Task created
Finished copy


In [ ]:
# Convert from geojson to a flat CSV structure
import json
import pandas as pd

data=json.load(open(b_file_path + '.geojson'))['features']

df_list = []
for f in data:
  geometry = f.get('geometry')
  #print(geometry)
  ids = f.get('id')
  props = f.get('properties')
  
  # select bounds
  bounds = props['bounds']
  del props['bounds']

  # select con hotspot feilds
  chs = props['con_hotspot_summary_km2']
  del props['con_hotspot_summary_km2']
  
  # select total carbon fields
  tmc = props['total_mangrove_carbon']
  #print(tmc)
  tmc_sel = [{y:{k:v for k, v in dict(tmc).get(y).items() if k in ('agb_co2e', 'agb_co2e', 'agb_co2e','toc_co2eha-1')}} for y in ['2016']][0]
  #print(tmc_sel)
  
  print(props)
  #out = pd.DataFrame(props)
  #print(out)
  
  #out['id'] = ids
  #out['geometry'] = json.dumps(geometry)
  #out['con_hotspot_summary_km2'] = json.dumps(chs)
  #df_list.append(out)
  
#df = pd.concat(df_list)
#df
#df = df.rename(columns={"type": "location_type", "length_coast_m": "coast_length_m", "id": "location_id", \
#                        'agb_mangrove_mgha-1': 'agb_mgha_1', 'hba_mangrove_m': 'hba_m', 'hmax_mangrove_m': 'hmax_m', \
#                       'mangrove_gain_m2': 'gain_m2', 'mangrove_loss_m2': 'loss_m2', 'length_mangrove_m': 'length_m' \
#                        ,'area_mangrove_m2': 'area_m2'})
#df
#df = df[['gain_m2', 'loss_m2', 'length_m', 'area_m2', 'hmax_m', 'agb_mgha_1', 'hba_m', 'location_id', 'location_type', 'name', 'con_hotspot_summary_km2']]
#df = df.reset_index()
#df
#df = df.rename(columns={"index": "date"})
#df
#df['date'] = pd.to_datetime(df['date'], format='%Y', errors='coerce')
#df.to_csv(b_file_path + '.csv', index=False, na_rep="NA", sep=";")

{'area_m2': 2273790826.4007034, 'coast_length_m': '263032.22', 'designation_eng': 'Ramsar Site, Wetland of International Importance', 'id': '2_000000000000000003d0', 'iso': 'AUS', 'iucn_cat': 'Not Reported', 'name': 'Cobourg Peninsula', 'perimeter_m': 749004.5878199834, 'type': 'wdpa', 'area_mangrove_gain_m2': {'2007': 2486874.72037162, '2008': 484482.17711133766, '2009': 33320.68683914484, '2010': 580698.905456543, '2015': 4120878.241110648, '2016': 1471791.3680048925}, 'area_mangrove_loss_m2': {'2007': 1024828.1143128638, '2008': 966950.3401644838, '2009': 355948.48839996936, '2010': 2150030.644890998, '2015': 101069.67422999961, '2016': 2571166.0984020755}, 'area_mangrove_m2': {'1996': 208262715.74178654, '2007': 206834215.03700858, '2008': 207306519.43899405, '2009': 207626201.60834336, '2010': 209202956.63380843, '2015': 205264844.22805315, '2016': 206276429.76463813}, 'agb_mangrove_hist_mgha-1': {'1996': [[0, 0], [10, 0], [20, 0], [30, 0], [40, 0], [50, 768.5803921568627], [60, 6

In [ ]:
df[['location_id']].size == pd.unique(df['location_id']).size

False

In [ ]:
# Send to GCS
import os
import subprocess
p_csv = b_file_path + '.csv'
p = 'gs://' + os.environ.get('MA_BUCKET') + '/' + bdir
subprocess.run(['gsutil', 'cp', p_csv, p])


CompletedProcess(args=['gsutil', 'cp', 'countries_data_2019085_ee_export.csv', 'gs://mangrove_atlas/ee-export-tables'], returncode=0)

In [ ]:
# View CSV
import pandas as pd
pd.read_csv(b_file_path + '.csv', sep=";")


,date,gain_m2,loss_m2,length_m,area_m2,hmax_m,agb_mgha_1,hba_m,location_id,location_type,name,con_hotspot_summary_km2
0,1996-01-01,0.000000e+00,0.000000e+00,65167.28,5.075309e+08,NaN,NaN,NaN,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
1,2000-01-01,NaN,NaN,NaN,NaN,19.92,260.21,12.62,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
2,2007-01-01,5.985117e+06,1.404493e+07,64694.98,4.994764e+08,NaN,NaN,NaN,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
3,2008-01-01,5.131899e+06,4.469590e+06,64148.45,5.001376e+08,NaN,NaN,NaN,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
4,2009-01-01,3.208974e+06,4.309829e+06,63901.10,4.990318e+08,NaN,NaN,NaN,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
5,2010-01-01,4.316326e+06,5.752221e+06,64280.28,4.975992e+08,NaN,NaN,NaN,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
6,2015-01-01,1.313068e+07,1.132364e+07,65776.31,4.994017e+08,NaN,NaN,NaN,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
7,2016-01-01,1.041104e+07,1.104505e+07,65467.70,4.987754e+08,NaN,NaN,NaN,1_1_00000000000000000000,aoi,Rufiji,"{""Monitoring Advised"": 31.2962, ""Requires Moni..."
8,1996-01-01,0.000000e+00,0.000000e+00,74792.05,5.777505e+08,NaN,NaN,NaN,1_2_00000000000000000000,aoi,Saloum,"{""Monitoring Advised"": 591.08, ""Requires Monit..."
9,2000-01-01,NaN,NaN,NaN,NaN,5.96,28.10,3.78,1_2_00000000000000000000,aoi,Saloum,"{""Monitoring Advised"": 591.08, ""Requires Monit..."
